In [43]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

pd.options.display.max_columns, pd.options.display.max_rows = 500, 200
pd.options.display.float_format = '{:.1f}'.format
import warnings

warnings.filterwarnings('ignore')
import joblib

from euro_soccer.preprocessing import DataPreprocessor
from euro_soccer.glicko_soccer import GlickoSoccer

In [44]:
matches = pd.read_csv('matches.csv')
matches.head()

,date,home_team,away_team,home_score,away_score,notes,country,league,season,file_path,tournament_type
0,05.11.2011,Kalju,Paide,1,2,NaN,Estonia,Meistriliiga,2011,first/Estonia_Meistriliiga_2011.csv,first
1,05.11.2011,Levadia,Kuressaare,4,0,NaN,Estonia,Meistriliiga,2011,first/Estonia_Meistriliiga_2011.csv,first
2,05.11.2011,Narva,Sillamae,2,3,NaN,Estonia,Meistriliiga,2011,first/Estonia_Meistriliiga_2011.csv,first
3,05.11.2011,Tammeka,Lasnamae,9,0,NaN,Estonia,Meistriliiga,2011,first/Estonia_Meistriliiga_2011.csv,first
4,05.11.2011,Viljandi,Flora,2,4,NaN,Estonia,Meistriliiga,2011,first/Estonia_Meistriliiga_2011.csv,first


In [56]:
matches = pd.read_csv('matches.csv')
matches = DataPreprocessor(is_actual_draw_predictions=False).preprocessing(matches)
matches.tail(5)

,index,date,home_team,away_team,country,league,season,tournament_type,tournament,outcome,is_pandemic,draw_probability
361296,59939,2022-08-31,Tuzla City,Siroki Brijeg,Bosnia And Herzegovina,Premier League,2022,1,Bosnia And Herzegovina. First,H,0,0.2
361295,59938,2022-08-31,Posusje,Zeljeznicar,Bosnia And Herzegovina,Premier League,2022,1,Bosnia And Herzegovina. First,A,0,0.2
361309,189174,2022-08-31,Plzen,Slovacko,Czech Republic,1. Liga,2022,1,Czech Republic. First,H,0,0.2
361301,189172,2022-08-31,Mlada Boleslav,Hradec Kralove,Czech Republic,1. Liga,2022,1,Czech Republic. First,A,0,0.2
361310,59937,2022-08-31,Igman K.,Sloboda,Bosnia And Herzegovina,Premier League,2022,1,Bosnia And Herzegovina. First,D,0,0.2


In [57]:
matches.loc[matches['home_team'].str.contains('Maccabi Haifa')].sort_values(['season'], ascending=False)

,index,date,home_team,away_team,country,league,season,tournament_type,tournament,outcome,is_pandemic,draw_probability
357497,183188,2022-05-21,Maccabi Haifa,Netanya,Israel,Ligat ha'Al - Championship Group,2021,1,Israel. First,H,0,0.2
349244,138891,2022-01-03,Maccabi Haifa,Maccabi Tel Aviv,Israel,Ligat ha'Al,2021,1,Israel. First,H,0,0.3
341754,138992,2021-09-11,Maccabi Haifa,Sakhnin,Israel,Ligat ha'Al,2021,1,Israel. First,H,0,0.2
343437,138975,2021-09-26,Maccabi Haifa,Maccabi Petah Tikva,Israel,Ligat ha'Al,2021,1,Israel. First,H,0,0.3
343940,138968,2021-10-03,Maccabi Haifa,H. Beer Sheva,Israel,Ligat ha'Al,2021,1,Israel. First,A,0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...
28104,199535,2011-04-23,Maccabi Haifa,Netanya,Israel,Ligat ha'Al - Championship Group,2010,1,Israel. First,A,0,0.2
28780,199531,2011-04-30,Maccabi Haifa,Hapoel Tel Aviv,Israel,Ligat ha'Al - Championship Group,2010,1,Israel. First,H,0,0.2
30930,199525,2011-05-16,Maccabi Haifa,Kiryat Shmona,Israel,Ligat ha'Al - Championship Group,2010,1,Israel. First,H,0,0.2
31781,374749,2011-05-25,Maccabi Haifa,Hapoel Tel Aviv,Israel,State Cup,2010,4,Israel. Cups,A,0,0.2


In [46]:
matches.shape

(328894, 13)

In [30]:
count_matches, count_seasons, count_tournaments, uefa_teams_without_national, duplicated_teams = DataPreprocessor().test_data(matches)

In [31]:
count_matches

,country,tournament_type,season,number_matches
57,Argentina,2,2021,11
540,Greece,2,2020,15


In [32]:
count_seasons

,country,tournament_type,number_seasons
20,Azerbaijan,2,2
80,Faroe Islands,2,2
103,Greece,2,2


In [33]:
count_tournaments

,country,number_tournaments,tournament_type


In [34]:
uefa_teams_without_national

[]

In [35]:
duplicated_teams

,home_team,country


In [ ]:
matches.info()

In [ ]:
# matches['draw_probability'].mean()

In [ ]:
# # matches.loc[(matches['home_team'].str.contains('B')) & (matches['away_team'].str.contains('AS Roma'))]

In [ ]:
# matches.info()

In [ ]:
# matches['season'].value_counts()

In [ ]:
# matches.loc[matches['tournament'].str.contains('Con')]

# Fit Params

In [23]:
glicko = GlickoSoccer()  #
league_params = glicko.fit_params(matches, 1, is_params_initialization=True)

Current Loss: 44728.34473780513
Russia. First 1500
Estonia. First 1500
Norway. Second 1500
Armenia. Second 1500
Slovakia. First 1500
Northern Ireland. Second 1500
Hungary. First 1500
Belgium. First 1500
Germany. First 1500
San Marino. First 1500
Ukraine. Second 1500
Scotland. First 1500
Bulgaria. First 1500
Serbia. First 1500
Argentina. Second 1500
Georgia. Second 1500
Azerbaijan. First 1500
Romania. Second 1500
Iceland. Second 1500
Norway. First 1500
Uruguay. First 1500
Wales. First 1500
Faroe Islands. First 1500
Italy. First 1500
Croatia. First 1500
Lithuania. Second 1500
Slovenia. Second 1500
Switzerland. First 1500
Brazil. Second 1500
England. First 1500
Scotland. Second 1500
Albania. First 1500
Kosovo. First 1500
Ireland. First 1500
Latvia. First 1500
Argentina. First 1500
Bosnia And Herzegovina. First 1500
Belgium. Second 1500
Brazil. First 1500
Finland. Second 1500
Ecuador. First 1500
Netherlands. Second 1500
Peru. First 1500
Malta. First 1500
Venezuela. First 1500
Albania. Seco

KeyboardInterrupt: 

# Ratings

In [54]:
glicko = GlickoSoccer()
league_params = joblib.load('data/league_params.pkl')
ratings = glicko.rate_teams(matches, league_params)
ratings = glicko.ratings_to_df(ratings, matches)
leagues_ratings = glicko.league_ratings(ratings, number_top_teams=10)
leagues_ratings

,#,league,rating
0,1,England. First,1889.1
1,2,Italy. First,1867.3
2,3,Spain. First,1849.4
3,4,Germany. First,1827.0
4,5,Cyprus. First,1801.3
5,6,Portugal. First,1778.8
6,7,France. First,1778.7
7,8,Czech Republic. First,1777.4
8,9,Northern Ireland. First,1775.1
9,10,Bulgaria. First,1773.3


In [ ]:
pd.concat([leagues_ratings[0:25].reset_index(drop=True),
           leagues_ratings[25:50].reset_index(drop=True)], axis=1)

In [ ]:
pd.concat([leagues_ratings[50:74].reset_index(drop=True),
           leagues_ratings[74:].reset_index(drop=True)], axis=1)

In [55]:
ratings

,#,team,rating,league
0,1,Crvena zvezda,2149.7,Serbia. First
1,2,Manchester City,2118.9,England. First
2,3,Flora,2115.8,Estonia. First
3,4,FC Porto,2111.1,Portugal. First
4,5,Levadia,2060.6,Estonia. First
...,...,...,...,...
2781,2782,Saldus/Brocenu,1090.9,NaN
2782,2783,Aktobe Zhas,1078.1,NaN
2783,2784,Cosmos,1059.4,San Marino. First
2784,2785,Balvu,1052.5,NaN


In [ ]:
leagues_ratings.to_csv('20.csv', index=False)

In [ ]:
ratings.head(50)

# Skellam

In [ ]:
# from euro_soccer.draw_model import DrawLightGBM

DrawLightGBM().cross_val_score()

In [ ]:
# import joblib
train = joblib.load('data/train.pkl')
train.head()

In [ ]:
# train.loc[train['team'] == 'Manchester City']

In [ ]:
# train['avg_scoring_10'].mean()